In [ ]:
!pip install opendatasets

In [ ]:
!pip install mysql-connector-python

Import necessary packages

In [2]:
import pandas as pd
import opendatasets as od
import mysql.connector as mysql
import csv
import json
from mysql.connector import Error

Function to convert CSV to JSON

In [3]:
def csvToJSON(csvFile, jsonFile):
    athletes_info = {}
    with open(csvFile, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
        for row in csvReader:
            key = row['S.NO']
            athletes_info[key] = row
    with open(jsonFile, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(athletes_info, indent=4))
         

Download Dataset From Kaggle Using API

In [4]:
url = 'https://www.kaggle.com/parulpandey/forbes-highest-paid-athletes-19902019/download'
od.download(url)


Skipping, found downloaded files in "./forbes-highest-paid-athletes-19902019" (use force=True to force download)


In [5]:
csv_filepath = '/Users/laurensmith/Documents/forbes-highest-paid-athletes-19902019/forbes_data.csv'
athletes_csv = pd.read_csv(csv_filepath)
athletes_csv

,S.NO,Name,Nationality,Current Rank,Previous Year Rank,Sport,Year,earnings ($ million)
0,1,Mike Tyson,USA,1,NaN,boxing,1990,28.6
1,2,Buster Douglas,USA,2,NaN,boxing,1990,26.0
2,3,Sugar Ray Leonard,USA,3,NaN,boxing,1990,13.0
3,4,Ayrton Senna,Brazil,4,NaN,auto racing,1990,10.0
4,5,Alain Prost,France,5,NaN,auto racing,1990,9.0
...,...,...,...,...,...,...,...,...
296,297,Stephen Curry,USA,6,9,Basketball,2020,74.4
297,298,Kevin Durant,USA,7,10,Basketball,2020,63.9
298,299,Tiger Woods,USA,8,11,Golf,2020,62.3
299,300,Kirk Cousins,USA,9,>100,American Football,2020,60.5


In [6]:
athletes_csv.drop('Previous Year Rank', axis=1)
athletes_csv = athletes_csv.where((pd.notnull(athletes_csv)), None)

Convert CSV to MySQL table

In [7]:
host = 'localhost'
username = 'root'
pwd = 'DS3002les1'

def csvToSQL(host, username, pwd):
    try:
        conn = mysql.connect(host=host, user=username,  
                        password=pwd, auth_plugin='mysql_native_password')
        
        # once connection is created, make sure table does not exist already and then create and populate it
        if conn.is_connected():
            cursor = conn.cursor()
            conn.cursor(buffered=True)
            cursor.execute("USE athletesDB;")
            record = cursor.fetchone()
            print("You're connected to database: ", record)
            cursor.execute('DROP TABLE IF EXISTS athletes;')
            print('Creating table....')
            cursor.execute("CREATE TABLE athletes (num INT NOT NULL, Name VARCHAR(100) NOT NULL, Nationality VARCHAR(250) NOT NULL, CurrentRank VARCHAR(100),PreviousYearRank VARCHAR(100), Sport VARCHAR(250), Year VARCHAR(10), earnings VARCHAR(100))")
            print("athletes table is created....")
            for i,row in athletes_csv.iterrows():
                sql = "INSERT INTO athletesDB.athletes VALUES (%s,%s,%s,%s,%s, %s, %s, %s)"
                cursor.execute(sql, tuple(row))
                conn.commit()
    except Error as e:
        print("Error connecting to MySQL", e)
        
    sql = "SELECT * FROM athletes"
    cursor.execute(sql)
    result = cursor.fetchall()
    for i in result:
        print(i)


Choose between converting from CSV to JSON and SQL and call previously created functions.

In [10]:
print("If you would like to convert from CSV to a SQL database, enter 'SQL'.")
print("If you would like to convert from CSV to JSON, enter 'JSON'.")
choice = input()
if(choice == "SQL"):
    csvToSQL(host, username, pwd)
    if(conn.is_connected()):
        cursor.close()
        conn.close()
elif(choice== "JSON"):
    csvFile = r'/Users/laurensmith/Documents/forbes-highest-paid-athletes-19902019/forbes_data.csv'
    jsonFile = r'/Users/laurensmith/Documents/forbes-highest-paid-athletes-19902019/forbes_data.json'
    csvToJSON(csvFile, jsonFile)
    athletes_json = pd.read_json(jsonFile)
else:
    print("Please enter a valid option.")

If you would like to convert from CSV to a SQL database, enter 'SQL'.
If you would like to convert from CSV to JSON, enter 'JSON'.
JSON


In [9]:
total_rows = len(athletes_csv)
total_columns = len(athletes_csv.columns)
print("Total number of rows in the dataset: " + str(total_rows))
print("Total number of columns in the dataset: " + str(total_columns))

Total number of rows in the dataset: 301
Total number of columns in the dataset: 8


Resources used:
- https://www.analyticsvidhya.com/blog/2021/04/how-to-download-kaggle-datasets-using-jupyter-notebook/

- https://stackoverflow.com/questions/63480004/import-a-csv-file-using-its-url-using-pandas-in-python
-https://www.geeksforgeeks.org/convert-csv-to-json-using-python/
-https://python.plainenglish.io/how-to-import-a-csv-file-into-a-mysql-database-using-python-script-791b051c5c33
-https://note.nkmk.me/en/python-pandas-len-shape-size/